**CenteredCanvas1080** is a simple Python tool that places each image from a selected folder onto a 1080x1080 white canvas at 72 DPI. Smaller images are centered without resizing. Larger images are scaled down so their largest dimension fits within 90% of the canvas, maintaining the original aspect ratio. All processed images are saved as PNGs in a png/ subfolder.

In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
import os
from PIL import Image

# Manually set your input directory here
input_dir = r"/content/drive/MyDrive/Photo Editing/Edited in Lightroom"

# Output folder will be created inside the input folder
output_dir = os.path.join(input_dir, "output")
os.makedirs(output_dir, exist_ok=True)

canvas_size = 1080
dpi = (72, 72)

for filename in os.listdir(input_dir):

    if filename == "output":
        continue

    if not filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
        continue

    img_path = os.path.join(input_dir, filename)
    with Image.open(img_path) as img:
        img = img.convert("RGBA")
        original_width, original_height = img.size

        canvas = Image.new("RGBA", (canvas_size, canvas_size), (255, 255, 255, 255))

        if original_width <= canvas_size and original_height <= canvas_size:
            x_offset = (canvas_size - original_width) // 2
            y_offset = (canvas_size - original_height) // 2
            canvas.paste(img, (x_offset, y_offset), img)
        else:
            max_dim = max(original_width, original_height)
            scale_factor = (canvas_size * 0.8) / max_dim
            new_width = int(original_width * scale_factor)
            new_height = int(original_height * scale_factor)
            resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            x_offset = (canvas_size - new_width) // 2
            y_offset = (canvas_size - new_height) // 2
            canvas.paste(resized_img, (x_offset, y_offset), resized_img)

        # Correctly closed string
        output_filename = os.path.splitext(filename)[0] + ".png"
        output_path = os.path.join(output_dir, output_filename)
        canvas.save(output_path, dpi=dpi)
